In [3]:
pwd

'/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC/notebook'

In [4]:
cd /mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN

/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN


/mnt/c/Users/nakagawa/Desktop/python_venv/feniax/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
cd ..

/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/PARAMETRIC


/mnt/c/Users/nakagawa/Desktop/python_venv/feniax/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os,sys
import numpy as np
sys.path.append('/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN/PARAMETRIC/')
from pch_parser import *

In [5]:
from pyNastran.bdf.bdf import read_bdf
from pyNastran.op2.op2 import read_op2

In [5]:
pwd

'/mnt/c/Users/nakagawa/Desktop/FEM4INAS/FEM4INAS/examples/BUG/NASTRAN'

In [6]:
NASTRAN_DIR="cmd.exe /c C:/MSC.Software/MSC_Nastran/20182/bin/nast20182.exe"
fname='./gafs/run_cao.bdf'
dir_output='./'
command=f'{NASTRAN_DIR} {fname} out={dir_output} old=no news=no'
os.system(command)

MSC Nastran V2018.2 (Intel Windows 10 Pro 6.2 9200) Fri Jul  5 20:35:05 2024
MSC Nastran beginning job run_cao.



*** SYSTEM INFORMATION MESSAGE (pgm: nastran, fn: estimate_job_requirements)
    Starting ESTIMATE, please wait...

*** USER INFORMATION MESSAGE (pgm: nastran, fn: estimate_job_requirements)
    Estimated memory=65370.0MB
    Estimated bpool=16342.5MB


MSC Nastran started  c:\msc.software\msc_nastran\20182\msc20182\win64i8\mscdate
MSC Nastran finished c:\msc.software\msc_nastran\20182\msc20182\win64i8\mscdate
MSC Nastran job run_cao completed.


0

In [6]:
import numpy as np
import scipy as sp
fname='./BUG_103cap.pch'
#def read_pch(fname,sparse=False):
f=open(fname,'r')
lines=f.read().splitlines()
f.close()
indice_k=[]
data_k=[]
indice_m=[]
data_m=[]
id1=0
nid_list=[]
for line in lines:
  if 'DMIG*   KAAX' in line:
    _line=line.split()
    id1=int(_line[2])*6+int(_line[3])-1
    if int(_line[2]) not in nid_list:
      nid_list.append(int(_line[2]))
    Flag_k=True
  elif 'DMIG*   MAAX' in line:
    _line=line.split()
    id1=int(_line[2])*6+int(_line[3])-1
    Flag_k=False
  elif '*' in line:
    _line=line.split()
    if len(_line)==3: #negative value
      dim=int(_line[-1][0])
      val=float(_line[-1][1:])
    elif len(_line)==4: #positive value
      dim=int(_line[-2])
      val=float(_line[-1])
    else: raise ValueError('Unknown format')
    id2=int(_line[1])*6+dim-1
    if Flag_k:
      indice_k.append([id1,id2])
      data_k.append(val)
    else:
      indice_m.append([id1,id2])
      data_m.append(val)
data_k=np.array(data_k)
data_m=np.array(data_m)
dim_original=np.repeat(nid_list,6)*6+np.repeat(np.arange(6),len(nid_list)).reshape(6,-1).T.flatten()
dim_condensed=np.arange(len(nid_list)*6)
dic_dim=dict(zip(dim_original,dim_condensed))
# replace indices
indice_k_=np.array([dic_dim[i] for i in np.array(indice_k).flatten()]).reshape(-1,2)
indice_m_=np.array([dic_dim[i] for i in np.array(indice_m).flatten()]).reshape(-1,2)
mat_shape=(len(nid_list)*6,len(nid_list)*6)
# make symmetric
msk_nondiag_k=(indice_k_[:,0]!=indice_k_[:,1])
indice_k=np.concatenate([indice_k_,indice_k_[msk_nondiag_k,::-1]],axis=0)
data_k=np.concatenate([data_k,data_k[msk_nondiag_k]],axis=0)
msk_nondiag_m=(indice_m_[:,0]!=indice_m_[:,1])
indice_m=np.concatenate([indice_m_,indice_m_[msk_nondiag_m,::-1]],axis=0)
data_m=np.concatenate([data_m,data_m[msk_nondiag_m]],axis=0)

Kaa=sp.sparse.coo_matrix((data_k,(indice_k[:,0],indice_k[:,1])),shape=mat_shape)
Maa=sp.sparse.coo_matrix((data_m,(indice_m[:,0],indice_m[:,1])),shape=mat_shape)
#  if not sparse:
#    Kaa=Kaa.toarray()
#    Maa=Maa.toarray()
#  return nid_list,Kaa,Maa

#nids,Ka,Ma=read_pch('./BUG_103cap.pch',sparse=True)

In [26]:
nids,Ka,Ma=read_pch('./BUG_103cap.pch',sparse=False)

In [ ]:
np.linalg.eigvalsh(Ma)

In [12]:
Kaa.sum(axis=0).min()

-436823992428959.0

In [33]:
np.array(Maa.sum(axis=0))[0]

array([ 6.29983419e+02,  6.29983419e+02,  6.29983419e+02, ...,
       -5.16494546e-04,  5.37891925e-04,  6.36063356e-08])

In [15]:
np.setdiff1d()

16076

In [23]:
nid_original=list(bdfmodel.nodes.keys())
nid_missing=np.setdiff1d(nid_original,nid_list)

In [34]:
len(nid_original)

16358

In [35]:
nid_missing

array([    1009,     2000,     2001,     2002,     2003,     2004,
           2005,     2006,     2007,     2008,     2009,     2010,
           2011,     2012,     2013,     2014,     2015,     2016,
           2017,     2018,     2019,     2020,     2021,     2022,
           2023,     2024,     2025,     2026,     2027,     2028,
           2029,     2030,     2031,     2032,     2033,     2034,
           2035,     2036,     2037,     2038,     2039,     2040,
           2041,     2042,     2043,     2044,     2045,     2046,
           2047,     2048,     2049,     2050,     2051,     2052,
           3009,    30010,    40000,    40001,    40002,    40003,
          40004,    40005,    40006,    40007,    40008,    40009,
          40010,    40011,    40012,    40013,    40014,    40015,
          40016,    40017,    40018,    40019,    40020,    40021,
          40022,    40023,    40024,    40025,    40026,    40027,
          40028,    40029,    40030,    40031,    40032,    40

In [39]:
import fem4inas.unastran.matrixbuilder as matrixbuilder

In [37]:
96444/6

16074.0

In [38]:
from pyNastran.op2.op2 import read_op2

In [78]:
op2model=read_op2('./bug_103cao.op2',debug=None)
eigvecs=op2model.eigenvectors[1].data
eigvals=op2model.eigenvectors[1].eigns

In [79]:
np.abs(np.array(nids)-op2model.eigenvectors[1].node_gridtype[:,0]).sum()

0

In [17]:
bdfmodel=read_bdf('./BUG_103cao.bdf',debug=None)

In [12]:
len(bdfmodel.nodes.keys())

16358

In [86]:
eigvec_f=eigvecs.reshape(eigvecs.shape[0],-1)

In [68]:
eigvec_f.shape

(100, 588)

In [99]:
import matplotlib.pyplot as plt

In [111]:
((Ma@eigvec_f.T)*eigvals).shape

(588, 100)

In [30]:
import plotly.graph_objects as go

In [151]:
mode_id=6
fig=go.Figure()
val=np.abs(((Ma@eigvec_f.T)*eigvals)[:,mode_id]/(Ka@eigvec_f.T)[:,mode_id]-1.0)
fig.add_trace(go.Scatter(x=np.arange(val.shape[0]),y=val))
#plt.plot(np.abs(((Ma@eigvec_f.T)*eigvals)[:,mode_id]/(Ka@eigvec_f.T)[:,mode_id]-1.0))
#plt.show()

/tmp/ipykernel_1157/1937592921.py:3: RuntimeWarning:

divide by zero encountered in divide



In [150]:
residual=np.abs(((Ma@eigvec_f.T)*eigvals)/(Ka@eigvec_f.T)-1.0)
residual[:,6]

/tmp/ipykernel_1157/4097205654.py:1: RuntimeWarning:

divide by zero encountered in divide



array([2.31152256e-01, 3.05013945e+00, 3.57099704e+00, 3.17739097e+01,
       9.96916798e-01, 1.00700247e+00, 4.01801580e-01, 5.19661939e-01,
       5.15439634e-01, 1.26112439e-01, 1.01320209e+00, 9.37690010e-01,
       2.16589453e-01, 6.19842550e+00, 9.03580333e+00, 2.42146944e-01,
       9.09025710e-02, 5.70563320e-01, 5.68450149e-02, 4.16151022e-01,
       3.19180265e-01, 8.92644699e-01, 9.81458994e-01, 6.61295893e-01,
       1.49762442e-02, 2.67388848e-01, 1.35698689e-02, 5.50912821e-02,
       9.54681370e-02, 8.00229792e-01, 2.14374987e-01, 3.98704544e-02,
       5.15509489e-03, 3.49177618e-02, 2.10706870e-01, 4.82036468e-01,
       4.26018982e+00, 2.14059831e+00, 5.05008784e-01, 2.45065146e-01,
       1.01564863e+00, 1.04313752e+00, 5.00830451e-01, 4.67606470e-01,
       3.93919812e+00, 5.08517620e-02, 1.01716867e+00, 1.03572719e+00,
       1.00756588e+00, 7.68750367e-01, 9.51699712e-01, 1.13044479e+00,
       9.99964751e-01, 9.29215861e-01, 8.37757056e-01, 1.57906214e-02,
      

In [117]:
residual=Ka@eigvec_f.T-(Ma@eigvec_f.T)*eigvals
norm_residual=np.linalg.norm(residual,axis=0)
print(norm_residual/np.linalg.norm(eigvec_f,axis=1))
#print(norm_residual)

[3.89588695e+06 1.01621957e+06 1.29016792e+06 1.72161386e+05
 3.21707685e+07 4.63370351e+07 4.80515224e+05 9.04958965e+04
 7.72097429e+05 7.64312648e+04 5.07389890e+06 3.08975749e+05
 3.27510947e+05 3.87668875e+05 2.44962150e+06 4.27868207e+05
 1.66977215e+06 5.33348267e+05 5.87522204e+06 6.84319592e+05
 9.58319637e+05 1.64246453e+06 9.83003420e+05 7.48649373e+04
 8.76826025e+05 1.32920726e+06 1.72140799e+06 2.05951848e+06
 6.71241983e+04 2.33240217e+05 1.63812340e+05 1.39049253e+05
 1.37426663e+05 4.25801557e+05 1.39170905e+06 2.79406912e+05
 9.11087813e+03 2.44037235e+04 1.26340995e+05 8.56741432e+04
 9.79000224e+05 1.95703388e+05 2.11503321e+05 1.89359517e+06
 3.05102968e+05 5.08483684e+04 5.63265111e+05 9.35202571e+04
 1.43594778e+05 1.91009739e+06 1.39423971e+06 6.56842286e+05
 1.49249773e+05 1.43579805e+05 5.40568771e+04 1.60038614e+05
 2.01734017e+04 3.37981665e+04 1.76570580e+05 1.16888890e+05
 2.41307494e+05 2.86340867e+05 5.92501020e+05 4.86072359e+04
 7.16407557e+04 6.846380

In [62]:
(Ma@eigvec_f.T)*eigvals

array([[-3.13302898e-22, -2.26400428e-21,  3.28979783e-19, ...,
        -9.59883571e-12, -7.97086899e-12,  1.02697264e-12],
       [ 6.19711094e-20, -1.86169319e-20,  1.19851184e-17, ...,
        -2.49665820e-11,  4.11650850e-11,  1.17729075e-12],
       [-6.05089761e-20, -1.19171723e-20, -8.00287518e-18, ...,
         5.89143316e-12,  8.86858741e-12, -1.77990187e-11],
       ...,
       [-1.01929971e-21, -8.76394408e-22, -2.10220450e-19, ...,
         3.45691464e-11,  1.71181133e-11,  1.35238213e-11],
       [-3.18023715e-21, -2.81955999e-21, -6.44955791e-19, ...,
         1.06648426e-10,  5.28298640e-11,  4.17290083e-11],
       [ 4.44231057e-22, -1.24257961e-21,  3.00219625e-19, ...,
        -3.81299034e-11, -1.85144741e-11, -1.47832527e-11]])

In [58]:
Ka@eigvec_f.T

array([[ 2.88268553e-14, -6.80412262e-15,  2.69462855e-13, ...,
        -7.37463197e-10, -8.39629505e-11,  5.89377428e-12],
       [-2.36609982e-13, -8.03935228e-14,  3.24147518e-14, ...,
         5.86041780e-10,  1.07535247e-09, -1.01125350e-11],
       [-6.18193328e-14, -2.58478595e-13,  5.44088073e-13, ...,
         7.72535040e-10,  2.01302378e-09,  2.21664319e-10],
       ...,
       [ 8.56085524e-13, -1.23977343e-13,  2.72555893e-12, ...,
        -1.18029890e-08, -3.21087642e-08,  3.92996259e-09],
       [ 1.53159059e-13, -3.50661709e-13, -1.57112924e-12, ...,
        -1.83089247e-08, -5.12843299e-09,  7.45700970e-11],
       [ 2.31189865e-12,  5.96069958e-12,  4.63189291e-11, ...,
         2.79387971e-07, -9.77191150e-08,  2.35431709e-08]])

In [9]:
import scipy as sp

In [84]:
eigval,eigvec=sp.linalg.eigh(Ma)

In [35]:
Ma.min()

-12741.94929

In [85]:
eigval

array([-5.09396614e-09, -5.09369792e-09, -1.34842537e-09, -1.34836484e-09,
       -1.77351467e-11, -1.75276178e-11, -1.83829062e-12, -1.81898940e-12,
        1.70530257e-13,  4.18491674e-13,  1.57875201e-12,  1.81898940e-12,
        3.63797881e-12,  2.03584705e-10,  2.03606021e-10,  2.03634734e-10,
        2.03636867e-10,  7.61190222e-10,  7.61207159e-10,  1.06525988e-09,
        1.06529897e-09,  1.18912169e-09,  1.18915892e-09,  1.32860123e-09,
        1.32872442e-09,  1.34292577e-09,  1.34305016e-09,  2.19338858e-09,
        2.19352385e-09,  2.78555490e-09,  2.78560168e-09,  9.77984340e-08,
        1.74322656e-07,  1.75344005e-07,  1.82748103e-07,  1.83375736e-07,
        1.84454108e-07,  1.91181022e-07,  2.11293148e-07,  2.26801307e-07,
        3.05916078e-02,  3.14416142e-02,  7.88986244e-02,  1.24866982e-01,
        1.82726124e-01,  3.54528796e-01,  3.61614709e-01,  3.70359448e-01,
        5.18235281e-01,  5.36121173e-01,  5.77947365e-01,  5.92497472e-01,
        7.15191422e-01,  

In [24]:
from pyNastran.bdf.bdf import read_bdf

In [26]:
bdfmodel=read_bdf('./BUG_103cap.bdf',debug=None)

In [32]:
np.sort(bdfmodel.asets[0].node_ids)[np.where(np.abs(Ma).sum(axis=1)==0.0)[0]//6]

array([    4001, 10004001])